In [207]:
import re, string, unicodedata
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', 40)

In [208]:
import contractions
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

import re, string, unicodedata
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\javier\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\javier\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\javier\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\javier\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Perfilamiento y entendimiento de datos

## 1. Lectura de datos

In [209]:
train_set = "fake_news_spanish.csv"
df = pd.read_csv(train_set, sep=';', encoding='utf-8')

print(df.shape)
df.head(3)

(57063, 5)


,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Euro...,El diario británico publicó este pas...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster so...,El valencianismo convoca en Castelló...,25/04/2022


## 2. Entendimiento de datos

TODO: Crear un diccionario de datos.

In [210]:
df.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

In [211]:
df.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Euro...,El diario británico publicó este pas...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster so...,El valencianismo convoca en Castelló...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC...","En política, igual que hay que negoc...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna traged...,"En una entrevista en El Punt Avui, e...",09/03/2018


In [212]:
copy = df.copy()

copy["Count"] = [len(text) for text in copy["Descripcion"]]
copy["Max"] = [ [max([len(word) for word in text.split(' ')])][0] for text in copy["Descripcion"] ]
copy["Min"] = [ [min([len(word) for word in text.split(' ')])][0] for text in copy["Descripcion"] ]
copy["Mean"] = [ np.mean([len(word) for word in text.split(' ')]) for text in copy["Descripcion"] ]

copy.head(3)

,ID,Label,Titulo,Descripcion,Fecha,Count,Max,Min,Mean
0,ID,1,'The Guardian' va con Sánchez: 'Euro...,El diario británico publicó este pas...,02/06/2023,218,13,1,5.083333
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,01/10/2023,3251,17,1,5.067164
2,ID,1,El 'Ahora o nunca' de Joan Fuster so...,El valencianismo convoca en Castelló...,25/04/2022,169,13,2,4.666667


### 2.1 Calidad de datos

### 2.1.1 Unicidad

In [213]:
copy[copy.duplicated()].shape

(445, 9)

In [214]:
copy.drop_duplicates(keep='first', inplace=True)
copy

,ID,Label,Titulo,Descripcion,Fecha,Count,Max,Min,Mean
0,ID,1,'The Guardian' va con Sánchez: 'Euro...,El diario británico publicó este pas...,02/06/2023,218,13,1,5.083333
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,01/10/2023,3251,17,1,5.067164
2,ID,1,El 'Ahora o nunca' de Joan Fuster so...,El valencianismo convoca en Castelló...,25/04/2022,169,13,2,4.666667
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC...","En política, igual que hay que negoc...",03/01/2022,133,15,2,5.700000
4,ID,0,Puigdemont: 'No sería ninguna traged...,"En una entrevista en El Punt Avui, e...",09/03/2018,242,14,1,4.651163
...,...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la ...,El gobierno regional han indicado qu...,08/06/2021,244,12,1,4.975610
57059,ID,0,El EQUO plantea ceder la presidencia...,Si la higiene democrática nos lleva ...,08/09/2020,314,11,1,5.300000
57060,ID,1,Alberto Garzón: 'Que los Borbones so...,El coordinador federal de IU asegura...,12/07/2018,70,11,2,5.454545
57061,ID,1,Vox exige entrar en el Gobierno de C...,Santiago Abascal: Vox tiene el derec...,13/02/2022,232,12,1,4.418605


In [215]:
# ¿Cómo manejar los titulos duplicados?
copy[copy.duplicated('Titulo', keep=False)].shape

(9952, 9)

In [216]:
# ¿Cómo manejar las descripciones duplicadas?
copy[copy.duplicated('Descripcion', keep=False)].shape

(13857, 9)

### 2.1.2 Completitud

In [217]:
copy.isna().sum()

ID              0
Label           0
Titulo         16
Descripcion     0
Fecha           0
Count           0
Max             0
Min             0
Mean            0
dtype: int64

In [218]:
# Reemplazar los NaN con la primera frase de la descripcion
copy \
    .loc[copy["Titulo"].isna(), "Titulo"] = copy["Descripcion"] \
    .apply(lambda t: t.split(".")[0] if isinstance(t, str) else t)

copy.isna().sum()

ID             0
Label          0
Titulo         0
Descripcion    0
Fecha          0
Count          0
Max            0
Min            0
Mean           0
dtype: int64

### 2.1.3 Validez

### 2.1.4 Consistencia

# 3. Preparación de datos

En esta fase se hará un proceso de tres etapas:

1. Limpieza de datos
2. Tokenización
3. Normalización

## 3.1 Limpieza de datos

In [219]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    return [word.lower() for word in words]

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def replace_numbers(words):
   """Replace all interger occurrences in list of tokenized words with textual representation"""
   p = inflect.engine()
   new_words = []
   for word in words:
       if word.isdigit():
           new_word = p.number_to_words(word)
           new_words.append(new_word)
           #print("if " + new_word)
       else:
           new_words.append(word)
   return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    stop_words = set(stopwords.words('spanish'))  # Cargar stopwords en español
    return [word for word in words if word not in stop_words]

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

## 3.2 Tokenización

In [232]:
#copy['Titulo'] = copy['Titulo'].apply(contractions.fix) No necesario, en español hay solo 2 contracciones
copy[['Titulo']].head(2)

,Titulo
0,'The Guardian' va con Sánchez: 'Euro...
1,REVELAN QUE EL GOBIERNO NEGOCIO LA L...


In [221]:
copy['words'] = copy['Titulo'].apply(word_tokenize)

In [222]:
copy.head(5)

,ID,Label,Titulo,Descripcion,Fecha,Count,Max,Min,Mean,words
0,ID,1,'The Guardian' va con Sánchez: 'Euro...,El diario británico publicó este pas...,02/06/2023,218,13,1,5.083333,"['The, Guardian, ', va, con, Sánchez..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,01/10/2023,3251,17,1,5.067164,"[REVELAN, QUE, EL, GOBIERNO, NEGOCIO..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster so...,El valencianismo convoca en Castelló...,25/04/2022,169,13,2,4.666667,"[El, 'Ahora, o, nunca, ', de, Joan, ..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC...","En política, igual que hay que negoc...",03/01/2022,133,15,2,5.700000,"[Iglesias, alienta, a, Yolanda, Díaz..."
4,ID,0,Puigdemont: 'No sería ninguna traged...,"En una entrevista en El Punt Avui, e...",09/03/2018,242,14,1,4.651163,"[Puigdemont, :, 'No, sería, ninguna,..."


In [223]:
copy['words'] = copy['words'].apply(preprocessing) #Aplica la eliminación del ruido

copy.sort_values(by="Max", ascending=False).head(3)

,ID,Label,Titulo,Descripcion,Fecha,Count,Max,Min,Mean,words
31324,ID,0,EXTRANJEROS QUE TENGAN MÁS DE *NUMBE...,EXTRANJEROS QUE TENGAN MÁS DE *NUMBE...,27/09/2024,3465,69,1,5.145390,"[extranjeros, mas, number, meses, me..."
15159,ID,0,Declarar a Bill Gates enemigo públic...,Declarar a Bill Gates enemigo públic...,02/08/2024,2669,55,1,5.297170,"[declarar, bill, gates, enemigo, pub..."
25104,ID,1,"""Esto antes era impensable"": la crec...",Chile se ha visto tradicionalmente c...,20/05/2024,14019,38,0,5.440055,"[impensable, creciente, preocupacion..."


## 3.3 Normalización

In [224]:
copy[["words"]].head(2)

,words
0,"[the, guardian, va, sanchez, europa,..."
1,"[revelan, gobierno, negocio, liberac..."


In [231]:
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

def stem_words(words):
    """Stem words in list of tokenized words"""
    return [stemmer.stem(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    return [lemmatizer.lemmatize(word, pos='v') for word in words]

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

#copy['words'] = 
copy['words'] = copy['words'].apply(stem_and_lemmatize) # Aplica lematización y Eliminación de Prefijos y Sufijos.
copy.head()

,ID,Label,Titulo,Descripcion,Fecha,Count,Max,Min,Mean,words
0,ID,1,'The Guardian' va con Sánchez: 'Euro...,El diario británico publicó este pas...,02/06/2023,218,13,1,5.083333,"[t, , h, , e, , , , g, , u, ,..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,01/10/2023,3251,17,1,5.067164,"[r, , e, , v, , e, , l, , , ,..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster so...,El valencianismo convoca en Castelló...,25/04/2022,169,13,2,4.666667,"[a, , h, , o, , r, , , , n, ,..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC...","En política, igual que hay que negoc...",03/01/2022,133,15,2,5.700000,"[i, , g, , l, , e, , s, , i, ,..."
4,ID,0,Puigdemont: 'No sería ninguna traged...,"En una entrevista en El Punt Avui, e...",09/03/2018,242,14,1,4.651163,"[p, , u, , i, , g, , d, , e, ,..."


## 3.4 Selección de campos

In [230]:
copy['words'] = copy['words'].apply(lambda x: ' '.join(map(str, x)))
copy.head(3)

,ID,Label,Titulo,Descripcion,Fecha,Count,Max,Min,Mean,words
0,ID,1,'The Guardian' va con Sánchez: 'Euro...,El diario británico publicó este pas...,02/06/2023,218,13,1,5.083333,t h e g u a r d v a s a n c h ...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,REVELAN QUE EL GOBIERNO NEGOCIO LA L...,01/10/2023,3251,17,1,5.067164,r e v e l g o b i e r n o n e g ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster so...,El valencianismo convoca en Castelló...,25/04/2022,169,13,2,4.666667,a h o r n u n c j o a n f u s ...


In [227]:
X_data, y_data = copy['words'], copy['Label']
y_data

0        1
1        0
2        1
3        1
4        0
        ..
57058    1
57059    0
57060    1
57061    1
57062    1
Name: Label, Length: 56618, dtype: int64

In [229]:
tf_idf = TfidfVectorizer()
X_tf_idf = tf_idf.fit_transform(copy['words'])
X_tf_idf.toarray()[0]

MemoryError: Unable to allocate 14.7 GiB for an array with shape (56618, 34944) and data type float64